In [47]:
#Importing the required libraries
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import models, layers

In [24]:
# getting the current working directory, and initializing image and label list
base_path = 'Dataset/Animals-10/raw-img'
images = []
labels = []

In [27]:
def load_images_from_subfolders(base_folder, target_size=(128, 128)):
    data = []
    labels = []
    for subdir, _, files in os.walk(base_folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
                img_path = os.path.join(subdir, file)
                img = Image.open(img_path)
                img = img.resize(target_size)
                img_array = np.array(img)
                if img_array.shape == (target_size[0], target_size[1], 3): 
                    data.append(img_array)
                    label = os.path.basename(subdir)
                    labels.append(label)
    return data, labels

images, labels = load_images_from_subfolders(base_path)
print(f"Loaded {len(images)} images.")

Loaded 26128 images.


In [32]:
images = np.array(images)
labels = np.array(labels)

In [33]:
print(f"Images shape: {images.shape}")
print(f"Labels shape: {labels.shape}")

Images shape: (26128, 128, 128, 3)
Labels shape: (26128,)


In [43]:
labels

array(['gallina', 'gallina', 'gallina', ..., 'elefante', 'elefante',
       'elefante'], dtype='<U10')

In [36]:
images = images.astype('float32') / 255.0
train_images, test_images, train_labels, test_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

In [38]:
encoder = LabelBinarizer()
train_labels = encoder.fit_transform(train_labels)
test_labels = encoder.transform(test_labels)

In [40]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

datagen.fit(train_images)

In [45]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


In [46]:
for layer in base_model.layers:
    layer.trainable = False

In [48]:
model = models.Sequential()
model.add(base_model)  
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5)) 
model.add(layers.Dense(len(encoder.classes_), activation='softmax'))

In [49]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [50]:
history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                    steps_per_epoch=len(train_images) // 32,
                    epochs=10,
                    validation_data=(test_images, test_labels))

Epoch 1/10


/Users/aryansaxena/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


653/653 ━━━━━━━━━━━━━━━━━━━━ 657s 1s/step - accuracy: 0.1685 - loss: 2.2967 - val_accuracy: 0.1891 - val_loss: 2.2208
Epoch 2/10
  1/653 ━━━━━━━━━━━━━━━━━━━━ 8:40 798ms/step - accuracy: 0.2188 - loss: 2.1710

2024-03-09 15:29:49.114906: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/contextlib.py:135: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(type, value, traceback)


653/653 ━━━━━━━━━━━━━━━━━━━━ 128s 196ms/step - accuracy: 0.2188 - loss: 1.0872 - val_accuracy: 0.1891 - val_loss: 2.2193
Epoch 3/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 662s 1s/step - accuracy: 0.1816 - loss: 2.2213 - val_accuracy: 0.1808 - val_loss: 2.2092
Epoch 4/10
  1/653 ━━━━━━━━━━━━━━━━━━━━ 9:49 904ms/step - accuracy: 0.2812 - loss: 2.2618

...

2024-03-09 15:56:11.848845: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


653/653 ━━━━━━━━━━━━━━━━━━━━ 130s 197ms/step - accuracy: 0.3438 - loss: 1.0764 - val_accuracy: 0.1808 - val_loss: 2.2093
Epoch 7/10
653/653 ━━━━━━━━━━━━━━━━━━━━ 656s 1s/step - accuracy: 0.1924 - loss: 2.2021 - val_accuracy: 0.1808 - val_loss: 2.2083
Epoch 8/10
  1/653 ━━━━━━━━━━━━━━━━━━━━ 8:41 799ms/step - accuracy: 0.1250 - loss: 2.3008

2024-03-09 16:09:17.196581: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]


653/653 ━━━━━━━━━━━━━━━━━━━━ 130s 198ms/step - accuracy: 0.1250 - loss: 1.1521 - val_accuracy: 0.1808 - val_loss: 2.2083
Epoch 9/10
196/653 ━━━━━━━━━━━━━━━━━━━━ 6:17 826ms/step - accuracy: 0.1861 - loss: 2.2101

KeyboardInterrupt: 